## Inputs

In [2]:
//
// Cairo Inputs
//

// Number of points to plot
const NUM_PTS: i32 = 20;

// Launch angle in degrees: -179 <= THETA_0_DEG <= +180
// Must be integer
const THETA_0_DEG: i32 = 105;

// Launch velocity magnitude: V_0 = ~100 is enough to reach top of plot area on vertical shot
const V_0: f64 = 100.0;

## Constants

In [3]:
//
// Math constants
//

// Number of terms in cosine Taylor series approximation
const N: i32 = 5;

// pi approximation to match that used in Cairo contract
const PI_APPROX: f64 = 3.141592654;


//
// Physical parameters
//

// Gravitational acceleration in m/s^2
const G: f64 = 9.8;

// Initial position
const X_0: f64 = 0.0;
const Y_0: f64 = 0.0;


//
// Plot parameters
//

// Min and max values for axes
const X_MAX: f64 = 1000.0;
const X_MIN: f64 = -X_MAX;
const Y_MAX: f64 = 500.0;
const Y_MIN: f64 = -Y_MAX;

## Math functions

In [4]:
//
// Math functions
//

// Factorial function
fn factorial(n: i32) -> f64 {
  let mut fact = 1.0;
  for i in 1..n + 1 {
    fact *= i as f64;
  }
  fact
}


// Cosine from Taylor series approximation
// Assume -pi <= theta <= +pi, so no need to shift theta
// Accuracy is improved if instead -pi/2 <= theta <= +pi/2
fn cosine_n_terms(theta: f64, n: i32) -> f64 {
  // n = number of terms (not order)
  // 2(n-1) = order
  // cosine(theta) ~= ((-1)^n)*(theta^(2n))/(2n)!
  //               ~= 1 - theta^2/2! + theta^4/4! - theta^6/6! + ...
  let mut cos_nth = 0.0;
  for i in 0..n {
    let neg_one: f64 = -1.0;
    let power_neg_one = neg_one.powi(i);
    let power_theta = theta.powi(2 * i);
    let fact = factorial(2 * i);
    cos_nth += power_neg_one * power_theta / fact;
  }
  cos_nth
}


// Cosine approximation
// Taylor series approximation is more accurate if -pi/2 <= theta_0 <= +pi/2. So:
// If theta_0 is in 2nd/3rd quadrant:
//   (1) move angle to 1st/4th quadrant for cosine approximation, and
//   (2) force negative sign for cosine(theta_0)
// If +90 or -90 degrees, use exact value of cos_theta_0
// (Use theta_0_deg for comparisons because calculated theta_0 in radians is slightly rounded)
// Then call cosine_n_terms
fn cosine_approx(theta_0: f64, theta_0_deg: i32, n: i32) -> f64 {
  match theta_0_deg {
    -179..=-91 => -cosine_n_terms(-PI_APPROX - theta_0, n),
    -90        => 0.0,
    -89..=89   => cosine_n_terms(theta_0, n),
    90         => 0.0,
    91..=180   => -cosine_n_terms(PI_APPROX - theta_0, n),
    _ => {
      println!("Error: theta_0_deg = {theta_0_deg} outside allowed range -179 to +180");
      0.0
    }
  }
}


// Sine approximation: need to force correct signs
fn sine_approx(theta_0: f64, cos_theta_0: f64) -> f64 {
  if theta_0 >= 0.0 {
    (1.0 - cos_theta_0.powi(2)).sqrt()
  } else {
    -((1.0 - cos_theta_0.powi(2)).sqrt())
  }
}

## Physics functions

In [5]:
//
// Physics functions
//
// Time of projectile in plot area
fn time_in_plot(
  theta_0_deg: i32,
  x_comps: (f64, f64),
  y_comps: (f64, f64, f64),
  plot_limits: (f64, f64, f64, f64)
) -> f64 {

  let (x_0, v_0x) = x_comps;
  let (y_0, v_0y, g) = y_comps;
  let (x_min, x_max, y_min, _y_max) = plot_limits;

  // Max time needed for y-direction
  let t_max_y = (v_0y + (v_0y.powi(2) - 2.0 * g * (y_min - y_0)).sqrt()) / g;

  // Find max time needed for x_direction, t_max_x
  // Then t_max is minimum of t_max_x and t_max_y
  let t_max_x: f64;
  let t_max: f64;

  if theta_0_deg.abs() == 90 {
    // abs(theta_0_deg) = 90, so v_0x = 0, so t_max_x = infinite, so
    t_max = t_max_y;
  } else {
    match theta_0_deg {
      // abs(theta_0_deg) < 90, so v_0x > 0, so projectile moves toward x_max
      -89..=89              => t_max_x = (x_max - x_0) / v_0x,
      // abs(theta_0_deg) > 90, so v_0x < 0, so projectile moves toward x_min
      -179..=-91 | 91..=180 => t_max_x = (x_min - x_0) / v_0x,
      _ => {
        println!("Error: theta_0_deg = {theta_0_deg} outside allowed range -179 to +180");
        t_max_x = 0.0;
      }
    }
    
    if t_max_x > t_max_y {
      t_max = t_max_y;
    } else {
      t_max = t_max_x;
    }
  }

  println!("t_max = {t_max}");
  t_max
}


fn x_value(x_comps: (f64, f64), t: f64) -> f64 {
  let (x_0, v_0x) = x_comps;
  x_0 + v_0x * t
}


fn y_value(y_comps: (f64, f64, f64), t: f64) -> f64 {
  let (y_0, v_0y, g) = y_comps;
  y_0 + v_0y * t - 0.5 * g * t.powi(2)
}

## Projectile calculations

In [6]:
// fn main() {

  // Check that THETA_0_DEG is within allowed range
  if THETA_0_DEG > 180 || THETA_0_DEG < -179 {
    println!("Error: THETA_0_DEG = {THETA_0_DEG} outside allowed range -179 to +180");
  }
  
  // Calculate theta in radians
  let theta_0 = THETA_0_DEG as f64 * PI_APPROX / 180.0;
  // Cosine approximation
  let cos_theta_0 = cosine_approx(theta_0, THETA_0_DEG, N);
  // Sine approximation
  let sin_theta_0 = sine_approx(theta_0, cos_theta_0);

  // Projectile velocity components in x- and y-directions
  let v_0x = V_0 * cos_theta_0;
  let v_0y = V_0 * sin_theta_0;

  println!("theta_0 = {theta_0}");
  println!("v_0x = {v_0x}");
  println!("v_0y = {v_0y}");
  
  // Tuples
  let x_comps: (f64, f64) = (X_0, v_0x);
  let y_comps: (f64, f64, f64) = (Y_0, v_0y, G);
  let plot_limits: (f64, f64, f64, f64) = (X_MIN, X_MAX, Y_MIN, Y_MAX);

  // Calculate time in plot
  let t_max = time_in_plot(THETA_0_DEG, x_comps, y_comps, plot_limits);
  let delta_t = t_max / ((NUM_PTS as f64) - 1.0);

// // Fill position vectors
//   let mut x_s = Vec::new();
//   let mut y_s = Vec::new();

//   for i in 0..NUM_PTS {
//     let t = i as f64 * delta_t;
//     let x = x_value(x_comps, t);
//     let y = y_value(y_comps, t);
//     x_s.push(x);
//     y_s.push(y);
//   }

// Fill position arrays
  let mut x_s: [f64; NUM_PTS as usize] = [0.0; NUM_PTS as usize];
  let mut y_s: [f64; NUM_PTS as usize] = [0.0; NUM_PTS as usize];

  for i in 0..NUM_PTS as usize {
    let t = i as f64 * delta_t;
    x_s[i] = x_value(x_comps, t);
    y_s[i] = y_value(y_comps, t);
  }

  // Need to use {:?}, Debug print, for printing Vec or Array
  // because {}, Display print, works only for single values
  println!("x_s = {x_s:?}");
  println!("y_s = {y_s:?}");

// }

theta_0 = 1.8325957148333336
v_0x = -25.882306286627742
v_0y = 96.59247497235584
t_max = 23.969806345067223
x_s = [0.0, -32.65230891864121, -65.30461783728242, -97.95692675592362, -130.60923567456484, -163.26154459320605, -195.91385351184724, -228.56616243048848, -261.2184713491297, -293.87078026777084, -326.5230891864121, -359.1753981050533, -391.8277070236945, -424.4800159423357, -457.13232486097695, -489.78463377961816, -522.4369426982594, -555.0892516169006, -587.7415605355417, -620.393869454183]
y_s = [0.0, 114.05942534793961, 212.52160460458768, 295.38653776994425, 362.6542248440092, 414.32466582678273, 450.39786071826467, 470.873809518455, 475.7525122273538, 465.0339688449611, 438.71817937127696, 396.8051438063011, 339.2948621500341, 266.187334402475, 177.48256056362447, 73.18054063348268, -46.718725387950826, -182.2152375006758, -333.3089957046923, -500.00000000000045]


## Plot with Plotters

In [7]:
:dep plotters = { version = "^0.3.4", default_features = false, features = ["evcxr", "all_series", "all_elements"] }
extern crate plotters;
use plotters::prelude::*;

In [8]:
evcxr_figure((640, 480), |root| {
    // The following code will create a chart context
    let mut chart = ChartBuilder::on(&root)
        .caption("Projectile path", ("Arial", 20).into_font())
        .x_label_area_size(40)
        .y_label_area_size(60)
        .build_cartesian_2d(X_MIN..X_MAX, Y_MIN..Y_MAX)?;
    
    chart.configure_mesh()
//         .disable_x_mesh()
//         .disable_y_mesh()
        .light_line_style(&WHITE)
        .x_desc("x (m)")
        .y_desc("y (m)")
        .draw()?;
    
    // `iter` method converts `x_s` and `y_s` each into an iterator
    // `zip` function combines these into a new iterator of ordered pairs
    let xy_s = x_s.iter().zip(y_s.iter());
    
    // `map` converts each ordered pair into a Circle to be plotted
    chart.draw_series(xy_s.map(|(x, y)| Circle::new((*x, *y), 3, BLUE.filled())));
    
//     let area = chart.plotting_area();
    
    Ok(())
}).style("width:100%")

Projectile path
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
y (m)
 
 
x (m)
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
-500.0
 
 
 
-400.0
 
 
 
-300.0
 
 
 
-200.0
 
 
 
-100.0
 
 
 
0.0
 
 
 
100.0
 
 
 
200.0
 
 
 
300.0
 
 
 
400.0
 
 
 
500.0
 
 
 
 
-1000.0
 
 
 
-800.0
 
 
 
-600.0
 
 
 
-400.0
 
 
 
-200.0
 
 
 
0.0
 
 
 
200.0
 
 
 
400.0
 
 
 
600.0
 
 
 
800.0
 
 
 
1000.0

## Plot with Plotly 
- but Markers error, so cannot get points w/o lines;
    - see https://github.com/igiagkiozis/plotly/issues/137)
- also much slower than Plotters

In [9]:
:dep ndarray = "0.15.6"
:dep plotly = { version = ">=0.7.0" }

In [10]:
extern crate ndarray;
extern crate plotly;
extern crate rand_distr;

In [11]:
use ndarray::Array;
use plotly::common::{Mode, Title};
use plotly::layout::{Axis, GridXSide, Layout, LayoutGrid};
use plotly::{Plot, Scatter};
use rand_distr::{num_traits::Float, Distribution};

// use plotly::{
//     color::{NamedColor, Rgb, Rgba},
//     common::{
//         ColorScale, ColorScalePalette, DashType, Fill, Font, Line, LineShape, Marker, Mode,
//         Orientation, Title,
//     },
//     layout::{Axis, BarMode, Layout, Legend, TicksDirection, TraceOrder},
//     sankey::{Line as SankeyLine, Link, Node},
//     Bar, Plot, Sankey, Scatter, ScatterPolar,
// };

In [12]:
// Must convert these to vectors for plotting
let x_s_vec: Vec<f64> = x_s.to_vec();
let y_s_vec: Vec<f64> = y_s.to_vec();

let trace = Scatter::new(x_s_vec, y_s_vec);
let mut plot = Plot::new();
plot.add_trace(trace);

let layout = Layout::new().height(525)
//     .grid(LayoutGrid::new()
//         .x_side(GridXSide::TopPlot))
    .title(Title::new("Projectile path"))
    .x_axis(Axis::new()
        .title(Title::new("x (m)"))
        .range(vec![X_MIN, X_MAX]))
    .y_axis(Axis::new()
        .title(Title::new("y (m)"))
        .range(vec![Y_MIN, Y_MAX]));

plot.set_layout(layout);

plot.notebook_display();